In [1]:
import yaml

with open("Contract.yaml") as stream:
     data = yaml.safe_load(stream)

In [2]:
data["dataset"]["environment"]

'PROD'

In [3]:
from Utils import add_data_set, extract_fileds_list

fields = extract_fileds_list( data)

In [4]:
# Inlined from /metadata-ingestion/examples/library/dataset_schema.py
# Imports for urn construction utility methods
from datahub.emitter.mce_builder import make_data_platform_urn, make_dataset_urn
from datahub.emitter.mcp import MetadataChangeProposalWrapper
from datahub.emitter.rest_emitter import DatahubRestEmitter

# Imports for metadata model classes
from datahub.metadata.schema_classes import (
    AuditStampClass,
    DateTypeClass,
    OtherSchemaClass,
    SchemaFieldClass,
    SchemaFieldDataTypeClass,
    SchemaMetadataClass,
    StringTypeClass,
    FixedTypeClass
)

schema_name = data["dataset"]["schemaName"]
platform = data["dataset"]["platform"]
data_set_name = data["dataset"]["schemaName"] + '.' + data["dataset"]["tableName"]
environment = data["dataset"]["environment"] 

event: MetadataChangeProposalWrapper = MetadataChangeProposalWrapper(
    entityUrn=make_dataset_urn(platform=platform, name=data_set_name, env=environment),
    aspect=SchemaMetadataClass(
        schemaName=schema_name,  # not used
        platform=make_data_platform_urn(platform),  # important <- platform must be an urn
        version=0,  # when the source system has a notion of versioning of schemas, insert this in, otherwise leave as 0
        hash="",  # when the source system has a notion of unique schemas identified via hash, include a hash, else leave it as empty string
        platformSchema=OtherSchemaClass(rawSchema="__insert raw schema here__"),
        lastModified=AuditStampClass(
            time=1640692800000, actor="urn:li:corpuser:ingestion"
        ),
        fields=fields,
    ),
)

# Create rest emitter
rest_emitter = DatahubRestEmitter(gms_server="http://localhost:8080")
rest_emitter.emit(event)
